<a href="https://colab.research.google.com/github/nklingen/CS-433-Project-1/blob/master/implementations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
from proj1_helpers import *

In [0]:
#from google.colab import files
#uploaded = files.upload()

# **1. Least Squares Gradient Descent**
Linear regression using gradient descent

In [2]:
def compute_gradient(y, tx, w):
    
    N = y.shape[0]
    gradient = (-1/N)*tx.T@(y-tx@w)
    
    return gradient

In [0]:
def least_squares_GD(y, tx, initial_w, max_iters, gamma): 
    ws = [initial_w]
    losses = []
    w = initial_w
    
    for n_iter in range(max_iters):
            
        # compute gradient and loss
        gradient = compute_gradient(y, tx, w)
        loss = compute_loss(y, tx, w)

        # update w by gradient
        w = w - gamma*gradient

        # store w and loss
        ws.append(w)
        losses.append(loss)
        print("Step " + str(n_iter + 1) + "/" + str(max_iters) + " loss = " + str(loss))
            
    return ws[-1], losses[-1]

# **2. Least Squares Stochastic Gradient Descent**
Linear regression using stochastic gradient descent

In [0]:
def least_squares_SGD(y, tx, initial_w, max_iters, gamma): 
    
    ws = [initial_w]
    losses = []
    w = initial_w
    
    for n_iter in range(max_iters):  
        for minibatch_y, minibatch_tx in batch_iter(y, tx, batch_size):
            
            # compute gradient and loss
            gradient = compute_stoch_gradient(minibatch_y, minibatch_tx, w)
            loss = compute_loss(minibatch_y, minibatch_tx, w)
            
            # update w by gradient
            w = w - gamma*gradient
            
            # store w and loss
            ws.append(w)
            losses.append(loss)
            
    return ws[len(ws)-1], losses[len(losses)-1]

# **3. Least Squares**
Least squares regression using normal equations

In [ ]:
# Mean Squared Error
def compute_loss(y, tx, w):
    
    MSE = 1/(2*y.shape[0])*np.sum(np.square(y-np.dot(tx,w)))
    
    return MSE

In [1]:
def least_squares(y, tx):
    
    a = tx.T@tx
    b = tx.T@y
    w = np.linalg.solve(a, b)
    
    return w

# **4. Ridge Regression**
Ridge regression using normal equations

In [0]:
def ridge_regression(y, tx, lambda_):
    
    a = (1/len(y))*(tx.T@tx) + 2*(lambda_*np.identity(tx.shape[1]))
    b = (1/len(y))*(tx.T@y)
    w = np.linalg.solve(a,b)
    
    MSE = compute_loss(y, tx, w)
    
    return w, MSE

# **5. Logistic Regression**
Logistic regression using gradient descent or SGD

In [ ]:
def sigmoid(t):
    sigmoid = math.exp(t)/(1+math.exp(t))
    return sigmoid


# def sigmoid(t):
#     """apply sigmoid function on t."""
#     return 1/(1+np.exp(-t))

def loss_logreg(y, tx, w):
    """compute the cost by negative log likelihood."""
    s = sigmoid(np.matmul(tx, w))
    loss = - np.sum((y*np.log(s) + (1-y)*np.log(1-s))/y.shape[0])
    return loss

# # compute the cost by negative log likelihood.
# def calculate_loss(y, tx, w):

#     loss1 = np.sum(np.log(1+math.exp(tx@w)))
#     loss2 = np.sum(y*tx@w)
#     loss = loss1-loss2
    
#     return loss

def gradient_logreg(y, tx, w):
    """compute the gradient of loss."""
    s = sigmoid(np.matmul(tx, w))
    return np.matmul(np.transpose(tx), s - y)

def grad_step_logreg(y, tx, w, gamma):
    """
    Do one step of gradient descen using logistic regression.
    Return the loss and the updated w.
    """
    loss = loss_logreg(y, tx, w)
    grad = gradient_logreg(y, tx, w)
    w = w - gamma * grad
    return loss, w

def logistic_regression(y, tx, initial_w, max_iters, gamma, batch_size=1):
    # init parameters
    threshold = 1e-8
    y = y.reshape(-1,1)
    w = initial_w.reshape(-1,1)
    loss=None
    # start the logistic regression
    for n_iter in range(max_iters):
        # get loss and update w.
        for minibatch_y, minibatch_tx in batch_iter(y, tx, batch_size):
            _, w = grad_step_logreg(minibatch_y, minibatch_tx, w, gamma)
        loss = loss_logreg(y, tx, w)
        # log info
        print("Log Regression({bi}/{ti}): loss={l}".format(
              bi=n_iter, ti=max_iters - 1, l=loss))
    return w, loss

# **6. Regularized Logistic Regression**
Regularized logistic regression using gradient descent
or SGD

In [ ]:
def penalized_logistic_regression(y, tx, w, lambda_):
    
    # compute the cost by negative log likelihood with added penalized term
    loss1 = np.sum(np.log(1+math.exp(tx@w)))
    loss2 = np.sum(y*tx@w)
    loss = loss1-loss2+lambda_/2*w.t@w
    
    gradient = calculate_gradient(y, tx, w) + lambda_*w
    
    hessian = calculate_hessian(y, tx,w) + 2*np.diag(np.ones(len(w)))
    
    return loss, gradient, hessian
    

In [0]:
def reg_logistic_regression(y, tx, lambda_, initial_w, max_iters, gamma):
    
    ws = [initial_w]
    losses = []
    hessian = []
    w = initial_w
    
    
    for n_iter in range(max_iters):
            
        l, g, h = penalized_logistic_regression(y, tx, w, lambda_)
        
        w = w - gamma*gradient
        
        # store w and loss
        ws.append(w)
        losses.append(loss)
        hessian.append(h)
        print("Step " + str(n_iter + 1) + "/" + str(max_iters) + " loss = " + str(loss))

    return loss[-1], gradient[-1], hessian[-1]